In [2]:
import sys
import pandas as pd
import re
pathModulesES = '../sauceforyall/'
sys.path.append(pathModulesES)
from yelpquery import YelpQuery
from pandasticsearch import Select
ye = YelpQuery()

In [3]:
import re
import ast

def fusion_attr(key, val):
    attr_full = ""
    key = key.replace("'","")
    sub_string = re.search('u\'(\w*)\'', val)
    if sub_string:
        attr_full = key + "_" + sub_string.group(1)
    else:
        val = val.replace("'","")
        attr_full = key + "_" + val
    return attr_full
    
def flatten_attr(nested_field):
    l=[]
    if nested_field:
        for (idx,val) in nested_field.items():
            f_level = idx
            s_level = re.findall(r"{(.+)}", val)    
            if s_level:
                for element in s_level[0].split(", "):
                    element_splited = element.split(":")
                    key_element = f_level + "_" + element_splited[0].strip()
                    val_element = element_splited[1].strip()
                    to_add_string = fusion_attr(key_element, val_element)
                    l.append(to_add_string)
            else:
                to_add_string = fusion_attr(f_level, val)
                l.append(to_add_string)    
    return ", ".join(l)

In [4]:
def getUserTaste(userID): #Op-cYCn71IJiIfx1IVjAwA
    mustArray = [
            ye.bodySingleTerm("user_id.keyword", userID),
            ye.bodyRange("date", "2016-01-01", "2018-12-31"),
        ]
    filterArray = []
    firstReview = ye.getComplexeQuery(index="yelp-review*", mustArray=mustArray, filterArray=filterArray, size=1)
    df_review = ye.getResultScrolling(firstReview)
    df_review = df_review[["user_id","business_id","stars"]]
    df_biz = pd.read_json("../dataset/business.json", lines=True)
    df_biz = df_biz[["business_id","attributes","categories"]]
    ### gather everything
    df = pd.merge(df_review,df_biz,how="inner", on='business_id')
    df['attributes'] = df['attributes'].map(lambda x: flatten_attr(x))
    user_average = df["stars"].mean()
    print(user_average)
    user_taste_cat = dict()
    user_taste_att = dict()
    for index, row in df.iterrows():
        rating_factor = 1 #if (row['stars'] > user_average) else 0
        #iterate categories
        if row['categories'] != None:
            categories = re.findall(r',?([\-\_\w]+),?', row['categories'])
            for c in categories:
                if(c in user_taste_cat):
                    user_taste_cat[c] = user_taste_cat[c] + 1 * rating_factor
                else:
                    user_taste_cat[c] = 1
        #iterate attributes
        if row['attributes'] != None:
            attributes = re.findall(r',?([\-\_\w]+),?', row['attributes'])
            for a in attributes:
                if(a in user_taste_att):
                    user_taste_att[a] = user_taste_att[a] + 1 * rating_factor
                else:
                    user_taste_att[a] = 1
    user_taste_cat = sorted(user_taste_cat.items(), key=lambda x: (-x[1], x[0]))
    user_taste_att = sorted(user_taste_att.items(), key=lambda x: (-x[1], x[0]))
    return user_taste_cat, user_taste_att

In [5]:
categories, attributes = getUserTaste("EC5nxNCWCmjHg1F14WrlxQ")

2.789473684210526


In [6]:
categories

[('Bars', 52),
 ('Restaurants', 51),
 ('Services', 43),
 ('Food', 28),
 ('Home', 22),
 ('Nightlife', 21),
 ('Medical', 20),
 ('American', 17),
 ('Cleaning', 15),
 ('Hair', 14),
 ('Health', 14),
 ('Beauty', 13),
 ('Spas', 13),
 ('Wine', 13),
 ('Pet', 12),
 ('Sushi', 12),
 ('Event', 11),
 ('Japanese', 11),
 ('New', 11),
 ('Coffee', 9),
 ('Italian', 9),
 ('Tea', 9),
 ('Bakeries', 8),
 ('Beer', 8),
 ('Breakfast', 8),
 ('Brunch', 8),
 ('Greek', 8),
 ('Planning', 8),
 ('Sandwiches', 8),
 ('Arts', 7),
 ('Care', 7),
 ('Doctors', 7),
 ('Estate', 7),
 ('Mediterranean', 7),
 ('Professional', 7),
 ('Salad', 7),
 ('Salons', 7),
 ('Cocktail', 6),
 ('Desserts', 6),
 ('Entertainment', 6),
 ('Office', 6),
 ('Plates', 6),
 ('Real', 6),
 ('Small', 6),
 ('Tapas', 6),
 ('Traditional', 6),
 ('Bar', 5),
 ('Centers', 5),
 ('Juice', 5),
 ('Massage', 5),
 ('Removal', 5),
 ('Seafood', 5),
 ('Shopping', 5),
 ('Skin', 5),
 ('Smoothies', 5),
 ('Venues', 5),
 ('Auto', 4),
 ('Automotive', 4),
 ('Cafes', 4),
 ('Cosmet

In [28]:
attributes

[('BusinessAcceptsCreditCards_True', 71),
 ('BusinessParking_validated_False', 57),
 ('RestaurantsGoodForGroups_True', 51),
 ('Ambience_divey_False', 50),
 ('Ambience_touristy_False', 49),
 ('Ambience_romantic_False', 48),
 ('Ambience_intimate_False', 46),
 ('BusinessParking_valet_False', 46),
 ('BusinessParking_street_False', 45),
 ('Ambience_upscale_False', 44),
 ('GoodForMeal_breakfast_False', 44),
 ('GoodForMeal_dinner_True', 43),
 ('RestaurantsAttire_casual', 43),
 ('Ambience_classy_False', 42),
 ('GoodForKids_True', 42),
 ('RestaurantsTakeOut_True', 42),
 ('Ambience_casual_True', 41),
 ('Ambience_hipster_False', 41),
 ('GoodForMeal_latenight_False', 40),
 ('RestaurantsPriceRange2_2', 40),
 ('NoiseLevel_average', 38),
 ('BikeParking_True', 37),
 ('BusinessParking_garage_False', 37),
 ('GoodForMeal_lunch_True', 37),
 ('RestaurantsTableService_True', 36),
 ('GoodForMeal_dessert_False', 35),
 ('RestaurantsDelivery_False', 35),
 ('WheelchairAccessible_True', 35),
 ('BusinessParking_lo

In [33]:
df_business = pd.read_json("../dataset/business.json", lines=True)
df_business['attributes'] = df_business['attributes'].map(lambda x: flatten_attr(x))
res = df_business[df_business.business_id == "L1-1P3acJc4gEFvWwjXcNQ"]

In [39]:
l = res.attributes.to_list()
for e in l[0].split(", "):
    print(e)

BusinessAcceptsCreditCards_True
OutdoorSeating_False
RestaurantsReservations_False
RestaurantsPriceRange2_2
Ambience_touristy_False
Ambience_hipster_False
Ambience_romantic_False
Ambience_divey_False
Ambience_intimate_False
Ambience_trendy_False
Ambience_upscale_False
Ambience_classy_False
Ambience_casual_True
RestaurantsTakeOut_True
RestaurantsTableService_False
RestaurantsDelivery_False
WiFi_free
Alcohol_beer_and_wine
GoodForKids_True
GoodForMeal_dessert_True
GoodForMeal_latenight_False
GoodForMeal_lunch_True
GoodForMeal_dinner_True
GoodForMeal_brunch_False
GoodForMeal_breakfast_False
HasTV_True
WheelchairAccessible_True
RestaurantsGoodForGroups_True
BusinessParking_garage_False
BusinessParking_street_False
BusinessParking_validated_False
BusinessParking_lot_False
BusinessParking_valet_False
RestaurantsAttire_casual
BikeParking_True
NoiseLevel_average
Caters_True


In [2]:
import pandas as pd
import re

def user_taste2(userID):
    firstReview = ye.getSingleMatch(index="yelp-review*", field="user_id", value=userID, size=10)
    df_review = ye.getResultScrolling(firstReview)
    #### get rated businesses info
    restaurants = df_review.business_id.unique()
    firstBiz = ye.getMultivalueTerm(index="yelp-business*", term="business_id.keyword", valueArray=restaurants, size=100)
    df_biz = ye.getResultScrolling(firstBiz)
    df_biz = df_biz[["business_id","name","city","state","categories","is_open"]]
    ### gather everything
    df = pd.merge(df_review,df_biz,how="inner", on='business_id')
    df = df[["business_id","name","stars","state","city","categories","is_open","text","review_id","date"]]
    ## filter restaurants only
    ##df = df[df.categories.map(lambda x: 'Restaurants' in x)]
    return df.sort_values(by='stars', ascending=False)


def user_taste(userID):
    df_business = pd.read_json("../dataset/business.json", lines=True)
    df_review = pd.read_json("../dataset/cleaned/review_cleaned_2016_2019_Vegas.json", lines=True)
    df_business = df_business[["business_id","name","city","state","categories","is_open","attributes"]]
    df_review = df_review[df_review["user_id"] == userID]
    ### gather everything
    df_business['attributes'] = df_business['attributes'].map(lambda x: flatten_attr(x))
    df = pd.merge(df_business,df_review,how="inner", on='business_id') 
    df = df[["business_id","name","stars","state","city","categories","attributes","is_open","text","review_id","date"]]
    ## filter restaurants only
    ##df = df[df.categories.map(lambda x: 'Restaurants' in x)]
    return df.sort_values(by='stars', ascending=False)


In [11]:
# d = user_taste("U4INQZOPSUaj8hMjLlZ3KA")
# pd.options.display.max_rows
# pd.set_option('display.max_colwidth', -1)
# d.head(20)